# LIHTC Financial Analysis - Interactive Notebook
## Comprehensive LIHTC Site Analysis with pyforma Integration

**Purpose**: Interactive analysis of LIHTC sites with real-time parameter adjustment and visualization

**Benefits of Using Jupyter**:
- 🔄 **Interactive Parameter Testing**: Adjust variables and see immediate results
- 📊 **Real-time Visualizations**: Charts and graphs update as you change parameters
- 📝 **Documentation**: Combine code, results, and explanations in one place
- 🎯 **Scenario Comparison**: Run multiple scenarios side-by-side
- 📈 **Sensitivity Analysis**: Interactive sliders for parameter exploration
- 💾 **Shareable Results**: Save analysis with outputs for stakeholder review

## 1. Setup and Imports

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
import sys
sys.path.append('/Users/williamrice/HERR Dropbox/Bill Rice/Structured Consultants/AI Projects/pyforma_integration/projects/TX_land_analysis')

from enhanced_lihtc_financial_model import (
    EnhancedLIHTCModel, 
    LIHTCFinancialParameters, 
    UnitMixParameters,
    DebtParameters,
    SoftFundsParameters,
    OperatingParameters
)

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")
print("🚀 Ready for interactive LIHTC analysis!")

## 2. Load Input Parameters from Excel

Load our comprehensive parameter file and display current settings

In [ ]:
# Load parameter file (update with your latest timestamp)
param_file = "/Users/williamrice/HERR Dropbox/Bill Rice/Structured Consultants/AI Projects/pyforma_integration/projects/TX_land_analysis/outputs/LIHTC_Input_Parameters_20250703_234629.xlsx"

# Load all parameter sheets
params = {}
sheet_names = ['LIHTC_Credits', 'Construction_Costs', 'Regional_Multipliers', 
               'Unit_Mix_Parameters', 'Debt_Parameters', 'Operating_Parameters']

for sheet in sheet_names:
    try:
        params[sheet] = pd.read_excel(param_file, sheet_name=sheet)
        print(f"✅ Loaded {sheet}: {len(params[sheet])} parameters")
    except Exception as e:
        print(f"❌ Error loading {sheet}: {e}")

# Display current LIHTC credit parameters
print("\n📊 Current LIHTC Credit Parameters:")
display(params['LIHTC_Credits'][['Parameter', 'Value', 'Description']].head())

## 3. Interactive Parameter Controls

Create interactive widgets to adjust key parameters in real-time

In [ ]:
# Create interactive parameter controls
print("🎛️ Interactive Parameter Controls")
print("Adjust parameters below and see immediate impact on analysis")

# Credit pricing slider
credit_pricing_widget = widgets.FloatSlider(
    value=0.85,
    min=0.70,
    max=1.00,
    step=0.01,
    description='Credit Pricing:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Construction cost slider
construction_cost_widget = widgets.FloatSlider(
    value=150.0,
    min=120.0,
    max=200.0,
    step=5.0,
    description='Construction Cost/SF:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Interest rate slider
interest_rate_widget = widgets.FloatSlider(
    value=5.5,
    min=3.0,
    max=8.0,
    step=0.1,
    description='Interest Rate (%):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# Vacancy rate slider
vacancy_rate_widget = widgets.FloatSlider(
    value=5.0,
    min=2.0,
    max=10.0,
    step=0.5,
    description='Vacancy Rate (%):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

# County selector
county_widget = widgets.Dropdown(
    options=['TRAVIS', 'HARRIS', 'DALLAS', 'BEXAR', 'WILLIAMSON', 'HAYS'],
    value='TRAVIS',
    description='County:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Display widgets
display(widgets.VBox([
    widgets.HTML("<h3>🎯 Key Financial Parameters</h3>"),
    credit_pricing_widget,
    construction_cost_widget,
    interest_rate_widget,
    vacancy_rate_widget,
    county_widget
]))

## 4. Real-Time Analysis Function

Define function that updates analysis based on widget values

In [ ]:
def run_interactive_analysis():
    """Run analysis with current widget values"""
    
    # Get current widget values
    credit_pricing = credit_pricing_widget.value
    construction_cost = construction_cost_widget.value
    interest_rate = interest_rate_widget.value / 100  # Convert to decimal
    vacancy_rate = vacancy_rate_widget.value / 100
    county = county_widget.value
    
    # Create custom parameters
    financial_params = LIHTCFinancialParameters(
        credit_pricing=credit_pricing,
        base_construction_cost_per_sf=construction_cost
    )
    
    debt_params = DebtParameters(
        permanent_interest_rate=interest_rate
    )
    
    operating_params = OperatingParameters(
        vacancy_rate=vacancy_rate
    )
    
    # Create model with custom parameters
    model = EnhancedLIHTCModel(
        financial_params=financial_params,
        debt_params=debt_params,
        operating_params=operating_params
    )
    
    # Sample site data for testing
    sample_site = {
        'acres': 10.0,
        'target_dua': 12,
        'county': county,
        'flood_zone': 'X',
        'market_type': 'Suburban',
        'qct_dda_eligible': True,
        'land_cost': 500000,
        'rent_1br_60pct': 988,
        'rent_2br_60pct': 1600,
        'rent_3br_60pct': 1370
    }
    
    # Run analysis
    result = model.calculate_sources_and_uses(sample_site)
    
    return result, model

print("✅ Interactive analysis function ready!")

## 5. Interactive Results Display

Run analysis and display results that update automatically when parameters change

In [ ]:
# Run analysis with current parameters
result, model = run_interactive_analysis()

# Display key results
print("📊 LIHTC FINANCIAL ANALYSIS RESULTS")
print("=" * 50)

# Extract key metrics
total_cost = result['cost_data']['total_development_cost']
cost_per_unit = result['cost_data']['cost_per_unit']
noi = result['income_data']['noi']
credit_4pct = result['credit_data']['credit_4pct_proceeds']
credit_9pct = result['credit_data']['credit_9pct_proceeds']
debt_capacity = result['debt_data']['net_loan_proceeds']
gap_4pct = result['funding_gap_4pct']
gap_9pct = result['funding_gap_9pct']

# Create results dataframe for display
results_df = pd.DataFrame({
    'Metric': [
        'Total Development Cost',
        'Cost per Unit',
        'Annual NOI',
        '4% Credit Proceeds',
        '9% Credit Proceeds',
        'Debt Capacity',
        '4% Funding Gap',
        '9% Funding Gap',
        '4% Feasible',
        '9% Feasible'
    ],
    'Value': [
        f"${total_cost:,.0f}",
        f"${cost_per_unit:,.0f}",
        f"${noi:,.0f}",
        f"${credit_4pct:,.0f}",
        f"${credit_9pct:,.0f}",
        f"${debt_capacity:,.0f}",
        f"${gap_4pct:,.0f}",
        f"${gap_9pct:,.0f}",
        "✅ Yes" if result['feasible_4pct'] else "❌ No",
        "✅ Yes" if result['feasible_9pct'] else "❌ No"
    ]
})

# Style the dataframe for better display
styled_df = results_df.style.set_properties(**{
    'background-color': '#f0f0f0',
    'color': 'black',
    'border-color': 'white'
})

display(styled_df)

## 6. Interactive Visualizations

Create charts that update automatically based on parameter changes

In [ ]:
# Create interactive visualization of sources and uses
def create_sources_uses_chart(result):
    """Create interactive sources and uses chart"""
    
    # Sources data for 4% scenario
    sources_4pct = {
        'LIHTC Credits': result['sources']['lihtc_4pct_proceeds'],
        'Permanent Loan': result['sources']['permanent_loan'],
        'HOME Funds': result['sources']['home_funds'],
        'Trust Funds': result['sources']['trust_funds'],
        'Deferred Dev Fee': result['sources']['deferred_developer_fee']
    }
    
    # Uses data
    uses = {
        'Hard Costs': result['uses']['hard_costs'],
        'Soft Costs': result['uses']['soft_costs'],
        'Land Cost': result['uses']['land_cost'],
        'Dev Fee Cash': result['uses']['cash_developer_fee'],
        'Construction Interest': result['uses']['construction_interest']
    }
    
    # Create subplots
    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'pie'}, {'type': 'pie'}]],
        subplot_titles=('Sources (4% Scenario)', 'Uses')
    )
    
    # Add sources pie chart
    fig.add_trace(
        go.Pie(
            labels=list(sources_4pct.keys()),
            values=list(sources_4pct.values()),
            name="Sources",
            marker_colors=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99', '#FF99CC']
        ),
        row=1, col=1
    )
    
    # Add uses pie chart
    fig.add_trace(
        go.Pie(
            labels=list(uses.keys()),
            values=list(uses.values()),
            name="Uses",
            marker_colors=['#FFB366', '#66FFB2', '#B366FF', '#FFD966', '#66D9FF']
        ),
        row=1, col=2
    )
    
    fig.update_layout(
        title_text="Project Sources and Uses Analysis",
        title_x=0.5,
        height=500
    )
    
    return fig

# Create and display the chart
sources_uses_fig = create_sources_uses_chart(result)
sources_uses_fig.show()

## 7. Parameter Sensitivity Analysis

Interactive analysis showing how changes in key parameters affect feasibility

In [ ]:
def sensitivity_analysis():
    """Run sensitivity analysis on key parameters"""
    
    # Parameter ranges to test
    credit_prices = np.arange(0.75, 1.00, 0.05)
    construction_costs = np.arange(130, 180, 10)
    interest_rates = np.arange(0.04, 0.08, 0.005)
    
    # Base site data
    base_site = {
        'acres': 10.0,
        'target_dua': 12,
        'county': 'TRAVIS',
        'flood_zone': 'X',
        'market_type': 'Suburban',
        'qct_dda_eligible': True,
        'land_cost': 500000,
        'rent_1br_60pct': 988,
        'rent_2br_60pct': 1600,
        'rent_3br_60pct': 1370
    }
    
    # Test credit pricing sensitivity
    credit_results = []
    for price in credit_prices:
        params = LIHTCFinancialParameters(credit_pricing=price)
        model = EnhancedLIHTCModel(financial_params=params)
        result = model.calculate_sources_and_uses(base_site)
        credit_results.append({
            'Credit_Pricing': price,
            'Gap_4PCT': result['funding_gap_4pct'],
            'Gap_9PCT': result['funding_gap_9pct'],
            'Feasible_4PCT': result['feasible_4pct'],
            'Feasible_9PCT': result['feasible_9pct']
        })
    
    credit_df = pd.DataFrame(credit_results)
    
    # Create sensitivity chart
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(
        x=credit_df['Credit_Pricing'],
        y=credit_df['Gap_4PCT'],
        mode='lines+markers',
        name='4% Funding Gap',
        line=dict(color='blue', width=3)
    ))
    
    fig.add_trace(go.Scatter(
        x=credit_df['Credit_Pricing'],
        y=credit_df['Gap_9PCT'],
        mode='lines+markers',
        name='9% Funding Gap',
        line=dict(color='red', width=3)
    ))
    
    # Add feasibility threshold line
    fig.add_hline(y=0, line_dash="dash", line_color="green", 
                  annotation_text="Feasibility Threshold")
    
    fig.update_layout(
        title="Credit Pricing Sensitivity Analysis",
        xaxis_title="Credit Pricing ($ per $1.00 credit)",
        yaxis_title="Funding Gap ($)",
        height=500,
        hovermode='x unified'
    )
    
    return fig, credit_df

# Run sensitivity analysis
sensitivity_fig, sensitivity_data = sensitivity_analysis()
sensitivity_fig.show()

print("\n📈 Credit Pricing Sensitivity Results:")
display(sensitivity_data.round(2))

## 8. Scenario Comparison

Compare multiple scenarios side-by-side

In [ ]:
def compare_scenarios():
    """Compare multiple scenarios side by side"""
    
    # Define scenarios
    scenarios = {
        'Base Case': {
            'credit_pricing': 0.85,
            'construction_cost': 150,
            'interest_rate': 0.055,
            'vacancy_rate': 0.05
        },
        'Conservative': {
            'credit_pricing': 0.80,
            'construction_cost': 165,
            'interest_rate': 0.065,
            'vacancy_rate': 0.07
        },
        'Optimistic': {
            'credit_pricing': 0.90,
            'construction_cost': 140,
            'interest_rate': 0.045,
            'vacancy_rate': 0.03
        }
    }
    
    # Base site
    base_site = {
        'acres': 10.0,
        'target_dua': 12,
        'county': 'TRAVIS',
        'flood_zone': 'X',
        'market_type': 'Suburban',
        'qct_dda_eligible': True,
        'land_cost': 500000,
        'rent_1br_60pct': 988,
        'rent_2br_60pct': 1600,
        'rent_3br_60pct': 1370
    }
    
    # Run each scenario
    comparison_results = []
    
    for scenario_name, params in scenarios.items():
        # Create model with scenario parameters
        financial_params = LIHTCFinancialParameters(
            credit_pricing=params['credit_pricing'],
            base_construction_cost_per_sf=params['construction_cost']
        )
        debt_params = DebtParameters(
            permanent_interest_rate=params['interest_rate']
        )
        operating_params = OperatingParameters(
            vacancy_rate=params['vacancy_rate']
        )
        
        model = EnhancedLIHTCModel(
            financial_params=financial_params,
            debt_params=debt_params,
            operating_params=operating_params
        )
        
        result = model.calculate_sources_and_uses(base_site)
        
        comparison_results.append({
            'Scenario': scenario_name,
            'Total_Cost': result['cost_data']['total_development_cost'],
            'NOI': result['income_data']['noi'],
            'Credit_4PCT': result['credit_data']['credit_4pct_proceeds'],
            'Credit_9PCT': result['credit_data']['credit_9pct_proceeds'],
            'Gap_4PCT': result['funding_gap_4pct'],
            'Gap_9PCT': result['funding_gap_9pct'],
            'Feasible_4PCT': '✅' if result['feasible_4pct'] else '❌',
            'Feasible_9PCT': '✅' if result['feasible_9pct'] else '❌'
        })
    
    comparison_df = pd.DataFrame(comparison_results)
    
    # Format for display
    display_df = comparison_df.copy()
    for col in ['Total_Cost', 'NOI', 'Credit_4PCT', 'Credit_9PCT', 'Gap_4PCT', 'Gap_9PCT']:
        display_df[col] = display_df[col].apply(lambda x: f"${x:,.0f}")
    
    return display_df

# Run scenario comparison
comparison_results = compare_scenarios()

print("🔄 Scenario Comparison Results:")
display(comparison_results)

## 9. Portfolio Analysis

Load and analyze multiple sites with current parameters

In [ ]:
# Load the 195 sites dataset
try:
    sites_file = "/Users/williamrice/HERR Dropbox/Bill Rice/Structured Consultants/AI Projects/CTCAC_RAG/code/FINAL_195_Sites_Complete_With_Poverty_20250621_213537.xlsx"
    sites_df = pd.read_excel(sites_file, sheet_name='All_195_Sites_Final')
    
    print(f"✅ Loaded {len(sites_df)} sites for portfolio analysis")
    
    # Show top 10 sites by existing economic score
    top_sites = sites_df.nlargest(10, 'Economic_Score')[['Address', 'County', 'Acres', 'Economic_Score']]
    
    print("\n🏆 Top 10 Sites by Existing Economic Score:")
    display(top_sites)
    
    # Create county distribution chart
    county_counts = sites_df['County'].value_counts().head(10)
    
    fig = px.bar(
        x=county_counts.index,
        y=county_counts.values,
        title="Site Distribution by County (Top 10)",
        labels={'x': 'County', 'y': 'Number of Sites'}
    )
    
    fig.update_layout(height=400)
    fig.show()
    
except Exception as e:
    print(f"❌ Error loading sites data: {e}")
    print("💡 Make sure the path to your 195 sites file is correct")

## 10. Export Results

Save current analysis and parameters for future reference

In [ ]:
from datetime import datetime
import os

def export_notebook_results():
    """Export current analysis results"""
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = "/Users/williamrice/HERR Dropbox/Bill Rice/Structured Consultants/AI Projects/pyforma_integration/projects/TX_land_analysis/outputs"
    
    # Create summary of current analysis
    current_params = {
        'Credit_Pricing': credit_pricing_widget.value,
        'Construction_Cost_SF': construction_cost_widget.value,
        'Interest_Rate_Pct': interest_rate_widget.value,
        'Vacancy_Rate_Pct': vacancy_rate_widget.value,
        'County': county_widget.value,
        'Analysis_Date': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Save current parameters
    params_df = pd.DataFrame([current_params])
    params_file = f"{output_dir}/Notebook_Analysis_Parameters_{timestamp}.xlsx"
    params_df.to_excel(params_file, index=False)
    
    print(f"✅ Exported current parameters to: {params_file}")
    print(f"📊 Current Analysis Summary:")
    display(params_df)

# Create export button
export_button = widgets.Button(
    description='Export Current Analysis',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

def on_export_click(b):
    export_notebook_results()

export_button.on_click(on_export_click)
display(export_button)

## 🎯 Summary: Jupyter Notebook Benefits for LIHTC Analysis

### ✅ **What This Notebook Provides**:

1. **🔄 Interactive Parameter Testing**: Real-time adjustment of credit pricing, costs, rates
2. **📊 Live Visualizations**: Charts that update automatically as you change parameters
3. **📈 Sensitivity Analysis**: See how parameter changes affect feasibility
4. **🔄 Scenario Comparison**: Compare multiple scenarios side-by-side
5. **📝 Documentation**: Combine analysis, code, and explanations in one place
6. **💾 Exportable Results**: Save analysis for stakeholder presentations

### 🚀 **Next Steps**:

1. **Install Required Packages**: `pip install plotly ipywidgets`
2. **Run This Notebook**: Execute cells to see interactive analysis
3. **Customize for Your Needs**: Add your specific parameters and sites
4. **Share with Stakeholders**: Export notebook as HTML for presentations

### 💡 **Future Enhancements**:

- **Real-time Market Data**: Connect to live LIHTC credit pricing feeds
- **Portfolio Optimization**: Interactive selection of best site combinations
- **Risk Modeling**: Monte Carlo simulation with probability distributions
- **Automated Reporting**: Generate investment memos from notebook analysis

**This interactive approach transforms LIHTC analysis from static calculations into dynamic, explorable financial modeling!** 🎉